In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [34]:
!pip install openai-whisper torch soundfile gtts joblib neattext seaborn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 3.6 MB/s eta 0:00:00


In [ ]:
pip install openai-whisper torch soundfile


In [35]:
from gtts import gTTS

# Define the text
text = "I am very happy today. This is a test for emotion detection."

# Convert text to speech
tts = gTTS(text, lang="en")

# Save the file
tts.save("test_audio.wav")

print("✅ Audio file 'test_audio.wav' generated successfully!")


✅ Audio file 'test_audio.wav' generated successfully!


In [36]:
import whisper

# Load Whisper model (small version to balance speed & accuracy)
whisper_model = whisper.load_model("small")  # Options: "tiny", "base", "small", "medium", "large"

print("Whisper model loaded successfully!")


/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


Whisper model loaded successfully!


In [28]:
!wget -O test_audio.wav https://www2.cs.uic.edu/~i101/SoundFiles/piano2.wav


--2025-03-19 13:22:43--  https://www2.cs.uic.edu/~i101/SoundFiles/piano2.wav
Resolving www2.cs.uic.edu (www2.cs.uic.edu)... 131.193.32.16
Connecting to www2.cs.uic.edu (www2.cs.uic.edu)|131.193.32.16|:443... failed: Connection timed out.
Retrying.

--2025-03-19 13:24:57--  (try: 2)  https://www2.cs.uic.edu/~i101/SoundFiles/piano2.wav
Connecting to www2.cs.uic.edu (www2.cs.uic.edu)|131.193.32.16|:443... failed: Connection timed out.
Retrying.

--2025-03-19 13:27:13--  (try: 3)  https://www2.cs.uic.edu/~i101/SoundFiles/piano2.wav
Connecting to www2.cs.uic.edu (www2.cs.uic.edu)|131.193.32.16|:443... failed: Connection timed out.
Retrying.

--2025-03-19 13:29:29--  (try: 4)  https://www2.cs.uic.edu/~i101/SoundFiles/piano2.wav
Connecting to www2.cs.uic.edu (www2.cs.uic.edu)|131.193.32.16|:443... failed: Connection timed out.
Retrying.

--2025-03-19 13:31:50--  (try: 5)  https://www2.cs.uic.edu/~i101/SoundFiles/piano2.wav
Connecting to www2.cs.uic.edu (www2.cs.uic.edu)|131.193.32.16|:443... 

In [37]:
import whisper

# Load Whisper model (small version for good balance)
whisper_model = whisper.load_model("small")

print("✅ Whisper model loaded successfully!")


✅ Whisper model loaded successfully!


In [38]:
def transcribe_audio(audio_path):
    """
    Convert speech to text using Whisper model.
    """
    result = whisper_model.transcribe(audio_path)
    return result["text"]

# Transcribe the test audio
audio_file = "test_audio.wav"
transcribed_text = transcribe_audio(audio_file)

print("🎙 Transcribed Text:", transcribed_text)


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


🎙 Transcribed Text:  I am very happy today. This is a test for emotion detection.


In [44]:
import os

# Check available files in the working directory
print("Files in the working directory:", os.listdir("/kaggle/working"))


Files in the working directory: ['.virtual_documents', 'test_audio.wav']


In [49]:
import pandas as pd
import neattext.functions as nfx

# Load dataset
df = pd.read_csv("/kaggle/input/emotions1/tweet_emotions.csv")  # Update with correct dataset path

# Rename columns if needed
df.rename(columns={'sentiment': 'Emotion', 'content': 'Text'}, inplace=True)

# Filter dataset to include only happy, sad, and angry
df = df[df['Emotion'].isin(['happy', 'sad', 'angry'])]

# Clean the text data
df['Clean_Text'] = df['Text'].apply(lambda x: nfx.remove_special_characters(x.lower()))
df['Clean_Text'] = df['Clean_Text'].apply(nfx.remove_stopwords)

# Display first few rows to verify
print(df.head())


Empty DataFrame
Columns: [tweet_id, Emotion, Text, Clean_Text]
Index: []


In [56]:
df = df[df['Emotion'].isin(['happy', 'sad', 'angry'])]


In [57]:
print("Dataset shape BEFORE filtering:", df.shape)  # Check data size before filtering
df = df[df['Emotion'].isin(['happy', 'sad', 'angry'])]
print("Dataset shape AFTER filtering:", df.shape)   # Check data size after filtering


Dataset shape BEFORE filtering: (0, 4)
Dataset shape AFTER filtering: (0, 4)


In [58]:
print("Column Names:", df.columns)


Column Names: Index(['tweet_id', 'Emotion', 'Text', 'Clean_Text'], dtype='object')


In [60]:
df.rename(columns={'sentiment': 'Emotion', 'content': 'Text'}, inplace=True)


In [61]:
df.columns = df.columns.str.strip()


In [62]:
print("Unique emotions:", df['Emotion'].unique())


Unique emotions: []


In [63]:
print(df.head())


Empty DataFrame
Columns: [tweet_id, Emotion, Text, Clean_Text]
Index: []


In [65]:
import os

dataset_path = "/kaggle/input/emotions1/tweet_emotions.csv"

# Check if the file exists and has content
if os.path.exists(dataset_path):
    print("✅ File found!")
    print("File size:", os.path.getsize(dataset_path), "bytes")
else:
    print("❌ File NOT found! Check your dataset path.")


✅ File found!
File size: 3768210 bytes


In [66]:
with open("/kaggle/input/emotions1/tweet_emotions.csv", "r") as file:
    for _ in range(5):  # Read and print first 5 lines
        print(file.readline().strip())


tweet_id,sentiment,content
1956967341,empty,@tiffanylue i know  i was listenin to bad habit earlier and i started freakin at his part =[
1956967666,sadness,Layin n bed with a headache  ughhhh...waitin on your call...
1956967696,sadness,Funeral ceremony...gloomy friday...
1956967789,enthusiasm,wants to hang out with friends SOON!


In [67]:
import pandas as pd

dataset_path = "/kaggle/input/emotions1/tweet_emotions.csv"

# Load dataset without headers (to inspect structure)
df = pd.read_csv(dataset_path, header=None)

# Print first few rows
print(df.head())


            0           1                                                  2
0    tweet_id   sentiment                                            content
1  1956967341       empty  @tiffanylue i know  i was listenin to bad habi...
2  1956967666     sadness  Layin n bed with a headache  ughhhh...waitin o...
3  1956967696     sadness                Funeral ceremony...gloomy friday...
4  1956967789  enthusiasm               wants to hang out with friends SOON!


In [70]:
import pandas as pd

# Load dataset and manually assign correct column names
dataset_path = "/kaggle/input/emotions1/tweet_emotions.csv"
df = pd.read_csv(dataset_path, names=["tweet_id", "Emotion", "Text"], skiprows=1)  # Skip first row

# Display first few rows after fixing
print(df.head())


     tweet_id     Emotion                                               Text
0  1956967341       empty  @tiffanylue i know  i was listenin to bad habi...
1  1956967666     sadness  Layin n bed with a headache  ughhhh...waitin o...
2  1956967696     sadness                Funeral ceremony...gloomy friday...
3  1956967789  enthusiasm               wants to hang out with friends SOON!
4  1956968416     neutral  @dannycastillo We want to trade with someone w...


In [71]:
df = df[df["Emotion"].isin(["happiness", "sadness", "anger"])]

# Check unique emotions after filtering
print("Filtered unique emotions:", df["Emotion"].unique())

# Display the first few rows
print(df.head())


Filtered unique emotions: ['sadness' 'happiness' 'anger']
     tweet_id  Emotion                                               Text
1  1956967666  sadness  Layin n bed with a headache  ughhhh...waitin o...
2  1956967696  sadness                Funeral ceremony...gloomy friday...
6  1956968487  sadness  I should be sleep, but im not! thinking about ...
8  1956969035  sadness            @charviray Charlene my love. I miss you
9  1956969172  sadness         @kelcouch I'm sorry  at least it's Friday?


In [69]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
import joblib

# Prepare data
X = df['Clean_Text']
y = df['Emotion']

# Convert text to TF-IDF vectors
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(X)

# Train a simple Naive Bayes model
model = MultinomialNB()
model.fit(X_tfidf, y)

# Save Model and Vectorizer
joblib.dump(model, "/kaggle/working/emotion_model.pkl")
joblib.dump(tfidf, "/kaggle/working/tfidf_vectorizer.pkl")

print("✅ Model and Vectorizer Saved Successfully!")


KeyError: 'Clean_Text'

In [53]:
import joblib
import neattext.functions as nfx

# Load trained model and vectorizer
emotion_model = joblib.load("emotion_model.pkl")
tfidf_vectorizer = joblib.load("tfidf_vectorizer.pkl")

print("✅ Emotion detection model loaded successfully!")


FileNotFoundError: [Errno 2] No such file or directory: 'emotion_model.pkl'

In [41]:
def predict_emotion(text):
    """
    Predict emotion from transcribed text.
    """
    # Preprocess text
    text_cleaned = nfx.remove_special_characters(text.lower())
    text_cleaned = nfx.remove_stopwords(text_cleaned)
    
    # Convert to numerical features using TF-IDF
    vectorized_text = tfidf_vectorizer.transform([text_cleaned])
    
    # Predict emotion
    prediction = emotion_model.predict(vectorized_text)
    
    return prediction[0]

# Predict emotion for transcribed text
predicted_emotion = predict_emotion(transcribed_text)

print("\n🎙 Speech-to-Text Output:", transcribed_text)
print("😊 Detected Emotion:", predicted_emotion)


NameError: name 'tfidf_vectorizer' is not defined

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a bar chart for emotion distribution
sns.barplot(x=["Happy", "Sad", "Angry"], y=[1 if predicted_emotion == "happy" else 0, 
                                            1 if predicted_emotion == "sad" else 0, 
                                            1 if predicted_emotion == "angry" else 0], 
            palette="coolwarm")

plt.title("Predicted Emotion")
plt.xlabel("Emotion")
plt.ylabel("Confidence")
plt.show()
